In [1]:
import os
os.chdir('../..')

In [2]:
from platform import python_version
print(python_version())

3.7.9


In [3]:
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from modules.datgan import DATWGAN

import networkx as nx
import json
import beepy

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
dataset = 'Chicago'
name = 'NOLINKS'

In [5]:
df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

In [6]:
df.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,23.42579,2,2,0,2,6,1,66,1.0,6,FTE,9.333333
1,drive,7,OTHER,1.71259,2,2,0,2,6,1,66,1.0,6,FTE,12.083333
2,drive,7,HOME_OTHER,21.77887,2,2,0,2,6,1,66,1.0,6,FTE,15.500000
3,drive,7,SHOPPING,2.02603,2,2,0,2,6,1,66,1.0,6,FTE,17.500000
4,drive,7,SHOPPING,0.87691,2,2,0,2,6,1,66,1.0,6,FTE,18.250000


In [7]:
continuous_columns = ["distance", "age", "departure_time"]

In [8]:
# personalised graph
graph = nx.DiGraph()

if name in ['FULLDAG', 'TRANSRED']:
    graph.add_edges_from([
        ("age", "license"),
        ("age", "education_level"),
        ("gender", "work_status"),
        ("education_level", "work_status"),
        ("education_level", "hh_income"),
        ("work_status", "hh_income"),
        ("hh_income", "hh_descr"),
        ("hh_income", "hh_size"),
        ("hh_size", "hh_vehicles"),
        ("hh_size", "hh_bikes"),
        ("work_status", "trip_purpose"),
        ("trip_purpose", "departure_time"),
        ("trip_purpose", "distance"),
        ("travel_dow", "choice"),
        ("distance", "choice"),
        ("departure_time", "choice"),
        ("hh_vehicles", "choice"),
        ("hh_bikes", "choice"),
        ("license", "choice"),
        # Non necessary links
        ("education_level", "hh_size"),
        ("work_status", "hh_descr"),
        ("work_status", "hh_size"),
        ("hh_income", "hh_bikes"),
        ("hh_income", "hh_vehicles"),
        ("trip_purpose", "choice")
    ])
    
    if name is 'TRANSRED':
        graph = nx.transitive_reduction(graph)
elif name is 'LINEAR':
    list_ = []
    for i in range(len(df.columns)-1):
        list_.append((df.columns[i], df.columns[i+1]))
    graph.add_edges_from(list_)
elif name is 'NOLINKS':
    for c in df.columns:
        graph.add_node(c)
elif name is 'CHOICE':
    list_ = []
    for c in df.columns:
        if c == 'choice':
            pass
        else:
            list_.append((c, 'choice'))
    graph.add_edges_from(list_)
else:
    print("PROBLEM")

In [9]:
output_folder = '../output/' + dataset + '/{}/'.format(name)

In [10]:
datgan = DATWGAN(continuous_columns, max_epoch=300, batch_size=500, output=output_folder, gpu=0)

In [11]:
test = datgan.fit(df, graph)

[1029 16:28:39 @DATWGAN.py:128] Preprocessing the data!
[1029 16:28:39 @data.py:387] Encoding categorical variable "choice"...
[1029 16:28:39 @data.py:387] Encoding categorical variable "travel_dow"...
[1029 16:28:39 @data.py:387] Encoding categorical variable "trip_purpose"...
[1029 16:28:39 @data.py:374] Encoding continuous variable "distance"...
[1029 16:28:39 @data.py:247]   Found 0 peaks!
[1029 16:28:39 @data.py:255]   Encoding with 10 components.
[1029 16:30:19 @data.py:274]   Predictions were done on 10 components => FINISHED!
[1029 16:30:19 @data.py:387] Encoding categorical variable "hh_vehicles"...
[1029 16:30:19 @data.py:387] Encoding categorical variable "hh_size"...
[1029 16:30:19 @data.py:387] Encoding categorical variable "hh_bikes"...


D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sklearn\mixture\_base.py:282: ConvergenceWarning: Initialization 10 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,


[1029 16:30:19 @data.py:387] Encoding categorical variable "hh_descr"...
[1029 16:30:19 @data.py:387] Encoding categorical variable "hh_income"...
[1029 16:30:19 @data.py:387] Encoding categorical variable "gender"...
[1029 16:30:19 @data.py:374] Encoding continuous variable "age"...
[1029 16:30:19 @data.py:247]   Found 6 peaks!
[1029 16:30:19 @data.py:255]   Encoding with 6 components.
[1029 16:31:18 @data.py:274]   Predictions were done on 6 components => FINISHED!
[1029 16:31:18 @data.py:387] Encoding categorical variable "license"...
[1029 16:31:18 @data.py:387] Encoding categorical variable "education_level"...
[1029 16:31:18 @data.py:387] Encoding categorical variable "work_status"...
[1029 16:31:18 @data.py:374] Encoding continuous variable "departure_time"...
[1029 16:31:19 @data.py:247]   Found 8 peaks!
[1029 16:31:19 @data.py:255]   Encoding with 8 components.


D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sklearn\mixture\_base.py:282: ConvergenceWarning: Initialization 10 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,


[1029 16:32:37 @data.py:274]   Predictions were done on 8 components => FINISHED!
[1029 16:32:37 @DATWGAN.py:141] Preprocessed data have been saved!


...


100%|###################################################################################|175/175[00:04<00:00,42.55it/s]

[1029 16:53:43 @base.py:285] Epoch 299 (global_step 78487) finished, time:4.11 seconds.
[1029 16:53:44 @saver.py:79] Model saved to ../output/Chicago/NOLINKS/model\model-78487.
[1029 16:53:44 @monitor.py:467] GAN_loss/d_loss: -0.075522
[1029 16:53:44 @monitor.py:467] GAN_loss/g_loss: -0.12539
[1029 16:53:44 @monitor.py:467] GAN_loss/kl_div: 0.11116
[1029 16:53:44 @monitor.py:467] QueueInput/queue_size: 50
[1029 16:53:44 @base.py:275] Start Epoch 300 ...



100%|###################################################################################|175/175[00:04<00:00,41.43it/s]

[1029 16:53:48 @base.py:285] Epoch 300 (global_step 78749) finished, time:4.22 seconds.
[1029 16:53:48 @saver.py:79] Model saved to ../output/Chicago/NOLINKS/model\model-78749.


[1029 16:53:48 @monitor.py:467] GAN_loss/d_loss: -0.073397
[1029 16:53:48 @monitor.py:467] GAN_loss/g_loss: -0.12725
[1029 16:53:48 @monitor.py:467] GAN_loss/kl_div: 0.10784
[1029 16:53:48 @monitor.py:467] QueueInput/queue_size: 50
[1029 16:53:48 @base.py:289] Training has finished!

[1029 16:53:48 @DATSGANModel.py:209] Creating cell for choice (in-edges: 0)
[1029 16:53:48 @DATSGANModel.py:209] Creating cell for travel_dow (in-edges: 0)
[1029 16:53:48 @DATSGANModel.py:209] Creating cell for trip_purpose (in-edges: 0)
[1029 16:53:48 @DATSGANModel.py:209] Creating cell for distance (in-edges: 0)
[1029 16:53:48 @DATSGANModel.py:209] Creating cell for hh_vehicles (in-edges: 0)
[1029 16:53:48 @DATSGANModel.py:209] Creating cell for hh_size (in-edges: 0)
[1029 16:53:48 @DATSGANModel.py:209] Creating cell for hh_bikes (in-edges: 0)
[1029 16:53:48 @DATSGANModel.py:209] Creating cell for hh_descr (in-edges: 0)
[1029 16:53:48 @DATSGANModel.py:209] Creating cell for hh_income (in-edges: 0)
[1029 

In [12]:
datgan.save('trained', force=True)

[1029 16:53:54 @DATSGAN.py:347] Model saved successfully.


In [13]:
beepy.beep(6)